# Another Possible Code for the Clusters

This code goes through the buffers one by one, rather than deconstructing all the data into one large array, like clusters_2.0 does.

First, we import Acqu (which parses the events?) and Timepix (which decodes the data??), as well as numpy, ROOT, and plotting classes of ROOT which allow us to organize and graph the data.

In [1]:
import Acqu as aq
import Timepix
import numpy as np
import ROOT
from rootpy.plotting import Hist, Hist2D, histogram, Canvas

Welcome to JupyROOT 6.16/00


Next, we import the data that was collected and open that data file.

In [2]:
inFile = '/w/work0/mainz/2019_05_Timepix3-Acqu/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Getting the data from Detector A:

In [58]:
TimepixAData = []

def plotTimepix():
    if(aq.epicsEvent==1):
        nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
        encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
        TimepixAData.append(Timepix.Decode(nHitsA,encodedA))

aq.runFunction(plotTimepix,0,50000)

#for m in range(len(TimepixAData)):
#    print(len(TimepixAData[m]))
print(len(TimepixAData[0]))

1492


A function that will help us sort the data by ToA:

In [4]:
def sortThird(val):     # a function to sort arrays of data by the third entry (in our case, ToA)
    return val[2]

A function that processes the clusters in detector A:

In [70]:
def process_cluster():
    start_index = cl[0]                  
    min_time = buff[start_index]['ToA']      
    meanx = 0.0                        
    meany = 0.0
    
    n = start_index
    while(n<start_index+cn):
        if(buff[n]['ToA'] < min_time):      
            min_time = buff[n]['ToA']       
        meanx+=buff[n]['x']              
        meany+=buff[n]['y']              
        n+=1                           
        
    meanx/=cn
    meany/=cn
    posA.Fill(meanx,meany)         
    multA.Fill(cn)
    
    n = start_index
    while(n<start_index+cn):                 
        dtime = buff[n]['ToA'] - min_time        
        dthistA.Fill(dtime)                
        dtime_toaA.Fill(dtime,buff[n]['ToT'])    
        n+=1                                 

A function for finding clusters in detector A:

In [77]:
def findclusters():
    
    global buff, cl, cn, blen, b, nc
    cl = [None]*100         
    cn = 0                 
    blen = 0                
    b = 0                   
    nc = 0                 
    

    
    for i in range(len(TimepixAData)):   

        b=0
        buff = []
        for k in range(len(TimepixAData[i])): 
            buff.append(TimepixAData[i][k])
        buff.sort(key=sortThird)
        blen = len(buff)
        
        while(b<blen):
            cn = 0        
            cl[cn] = b        
            cn+=1        

       
            c=b+1     
            while((c<b+99) and (c<blen)):
                if(abs(buff[c]['ToA']- buff[b]['ToA'] < 100)):
                    cl[cn] = c                          
                    cn+=1
                else:
                    process_cluster()   
                    b+=cn                  
                    cn = 0                     
                    cl = [None]*100         
                    nc+=1                          

                    if(c<b+99 and c<blen):       
                        cl[cn] = b                
                        cn+=1                       
                c+=1 # end of inner while loop
                if(((c>=blen) and cn!=0)or((c>=b+99) and cn!=0)): # if you're exiting the loop but have a cluster
                    process_cluster()
                    b+=cn                  
                    cn = 0                     
                    cl = [None]*100         
                    nc+=1
                    
            b+=1
            
            
        cl = [None]*100
        

Running and graphing the data for detector A:

In [78]:
ROOT.enableJSVis()
c1 = ROOT.TCanvas("c1","Clusters by Buffers", 1000, 1000)
c1.Divide(2,2)

posA = Hist2D(256,0,256,256,0,256)
multA = Hist(50,0,50)
dthistA = Hist(100,0,100)
dtime_toaA = Hist2D(100,0,100,230,0,230)

findclusters()

c1.cd(1)
posA.Draw("colz")
c1.cd(2)
multA.Draw()
c1.cd(3)
dthistA.Draw()
c1.cd(4)
dtime_toaA.Draw("colz")
c1.Draw()
